In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import sys
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Circle
import seaborn as sns; 
from IPython.display import HTML

In [3]:
#config parser
import configparser

sys.path.insert(0, '../Common/')
from AISDataManager import AISDataManager
import Constants as c
import HMUtils as hMUtil
import TimeUtils as timeUtils
from GridClass import GridClass

#MyConfig.INI stores all the run time constants
config = configparser.ConfigParser()
config.read('../MyConfig.INI')

from joblib import Parallel, delayed
import multiprocessing
aISDM = AISDataManager()

In [4]:
lonMin = (float)(config['REGEION']['LON_MIN'])
lonMax = (float)(config['REGEION']['LON_MAX'])

latMin = (float)(config['REGEION']['LAT_MIN'])
latMax = (float)(config['REGEION']['LAT_MAX'])

print(lonMin,latMin)
print(lonMax,latMax)

increStep = (float)(config['HEATMAP']['INCR_STEP'])
incrRes = (int)(config['HEATMAP']['INCR_RES'])

-120.0 31.0
-117.0 34.5


In [5]:
heatMapGrid = hMUtil.generate_grid(lonMin, lonMax, latMin, latMax, increStep, incrRes)
boundaryArray = heatMapGrid[2]
horizontalAxis = heatMapGrid[0]
verticalAxis = heatMapGrid[1]

In [6]:
#first load all the arrays
#i.e from 40 to 48
#and we sum every element, we get
probDir = "../Data/AIS_0117_1217_31_M120_345_M117/Transition/"

tPList = []

for i in range(9):
    probFile = probDir + '4' + str(i) + '.npy'
    probVal = np.load(probFile)
    tPList.append(probVal.copy()) 

#now sum of probabilities will be 1
sum44 = np.zeros(tPList[0].shape)
for i in range(len(tPList)):
    sum44 = sum44 + tPList[i]

In [7]:
print(len(boundaryArray))

1050


In [8]:
#now generate transition probability tabel
#using boundary array
gridList = []

for i in range(len(boundaryArray)):
# for i in range(2):
#     print(boundaryArray[i])
    #left top transition probabilities
    gridIdx = boundaryArray[i][4]
    tempLonMin = boundaryArray[i][0]
    tempLonMax = boundaryArray[i][1]
    tempLatMin = boundaryArray[i][2]
    tempLatMax = boundaryArray[i][3]
    
    gridList.append(GridClass(gridIdx))
    
    gridList[-1].set_lon_min_max(tempLonMin,tempLonMax)
    gridList[-1].set_lat_min_max(tempLatMin,tempLatMax)
    
    #takes care of divide by zero condition
    if(sum44[gridIdx] > 0):
        p40 = tPList[0][gridIdx]/sum44[gridIdx]
        p41 = tPList[1][gridIdx]/sum44[gridIdx]
        p42 = tPList[2][gridIdx]/sum44[gridIdx]
        p43 = tPList[3][gridIdx]/sum44[gridIdx]
        p44 = tPList[4][gridIdx]/sum44[gridIdx]
        p45 = tPList[5][gridIdx]/sum44[gridIdx]
        p46 = tPList[6][gridIdx]/sum44[gridIdx]
        p47 = tPList[7][gridIdx]/sum44[gridIdx]
        p48 = tPList[8][gridIdx]/sum44[gridIdx]
    
        gridList[-1].set_p40(p40)
        gridList[-1].set_p41(p41)
        gridList[-1].set_p42(p42)
        gridList[-1].set_p43(p43)
        gridList[-1].set_p44(p44)
        gridList[-1].set_p45(p45)
        gridList[-1].set_p46(p46)
        gridList[-1].set_p47(p47)
        gridList[-1].set_p48(p48)

In [10]:
p40Probs = []
p41Probs = []
p42Probs = []
p43Probs = []
p44Probs = []
p45Probs = []
p46Probs = []
p47Probs = []
p48Probs = []

for grid in gridList:
    p40Probs.append(grid.p40)
    p41Probs.append(grid.p41)
    p42Probs.append(grid.p42)
    p43Probs.append(grid.p43)
    p44Probs.append(grid.p44)
    p45Probs.append(grid.p45)
    p46Probs.append(grid.p46)
    p47Probs.append(grid.p47)
    p48Probs.append(grid.p48)

In [11]:
p40ProbsNP = np.array(p40Probs)
p41ProbsNP = np.array(p41Probs)
p42ProbsNP = np.array(p42Probs)
p43ProbsNP = np.array(p43Probs)
p44ProbsNP = np.array(p44Probs)
p45ProbsNP = np.array(p45Probs)
p46ProbsNP = np.array(p46Probs)
p47ProbsNP = np.array(p47Probs)
p48ProbsNP = np.array(p48Probs)

In [16]:
print(np.mean(p40ProbsNP))
print(np.median(p40ProbsNP))
print(np.min(p40ProbsNP))
print(np.max(p40ProbsNP))
print(np.var(p40ProbsNP))

0.06805391286238806
0.0476194211310691
0.0
1.0
0.007293107561779576


In [17]:
print(np.mean(p41ProbsNP))
print(np.median(p41ProbsNP))
print(np.min(p41ProbsNP))
print(np.max(p41ProbsNP))
print(np.var(p41ProbsNP))

0.08724778770792514
0.07385856448573414
0.0
1.0
0.007808572580199126


In [18]:
print(np.mean(p42ProbsNP))
print(np.median(p42ProbsNP))
print(np.min(p42ProbsNP))
print(np.max(p42ProbsNP))
print(np.var(p42ProbsNP))

0.053745200797753946
0.0315917066779896
0.0
1.0
0.007056445765791615


In [19]:
print(np.mean(p43ProbsNP))
print(np.median(p43ProbsNP))
print(np.min(p43ProbsNP))
print(np.max(p43ProbsNP))
print(np.var(p43ProbsNP))

0.08798283694182002
0.08123871841155235
0.0
1.0
0.0056850877354772365


In [20]:
print(np.mean(p44ProbsNP))
print(np.median(p44ProbsNP))
print(np.min(p44ProbsNP))
print(np.max(p44ProbsNP))
print(np.var(p44ProbsNP))

0.2809076117927554
0.2700519278670976
0.0
1.0
0.033027800765329164


In [21]:
print(np.mean(p45ProbsNP))
print(np.median(p45ProbsNP))
print(np.min(p45ProbsNP))
print(np.max(p45ProbsNP))
print(np.var(p45ProbsNP))

0.10862052873319518
0.10388384042774979
0.0
1.0
0.009277263549837277


In [22]:
print(np.mean(p46ProbsNP))
print(np.median(p46ProbsNP))
print(np.min(p46ProbsNP))
print(np.max(p46ProbsNP))
print(np.var(p46ProbsNP))

0.04662423144858971
0.02162996693905558
0.0
1.0
0.006879599451204785


In [23]:
print(np.mean(p47ProbsNP))
print(np.median(p47ProbsNP))
print(np.min(p47ProbsNP))
print(np.max(p47ProbsNP))
print(np.var(p47ProbsNP))

0.08202601837086665
0.0779917021122627
0.0
1.0
0.0058311955804541075


In [24]:
print(np.mean(p48ProbsNP))
print(np.median(p48ProbsNP))
print(np.min(p48ProbsNP))
print(np.max(p48ProbsNP))
print(np.var(p48ProbsNP))

0.06764901420184875
0.05280270462039317
0.0
1.0
0.005629198037996383
